In [ ]:
# !jupyter nbconvert --to script network_maude_sim.ipynb

In [67]:
%matplotlib inline

import pandas as pd
import re
from IPython.display import Image, display, Markdown, HTML
import copy
from pprint import pprint
# import utils.helpers as helpers
import networkx as nx
import traceback, sys
from abc import ABC, abstractmethod, ABCMeta

import numpy as np
import matplotlib.pyplot as plt

from utils.utils import * # for functions `process_subtypes`, `filter_fun`, `init_logger`

In [2]:
# Start a log file (from utils module)
# init_logger(filename="supply_demand")

In [3]:
# Run this from the the /pycims_prototype/ directory
import Reader
import Model

file = '../../pyCIMS_model_description.xlsm'

# Create a model description reader
my_reader = Reader.Reader(infile=file,
                          sheet_map={'model': 'Model',
                                     'incompatible': 'Incompatible',
                                     'default_tech': 'Technologies'},
                          node_col='Node')


# Create a model from the reader
my_model = Model.Model(my_reader)

# Build the model graph
my_model.build_graph()



pyCIMS
pyCIMS.Canada
pyCIMS.Canada.Alberta
pyCIMS.Canada.Alberta.Residential
pyCIMS.Canada.Alberta.Residential.Buildings
pyCIMS.Canada.Alberta.Residential.Buildings.Shell
pyCIMS.Canada.Alberta.Residential.Buildings.Shell.Space heating
pyCIMS.Canada.Alberta.Residential.Buildings.Shell.Space heating.Furnace
pyCIMS.Canada.Alberta.Residential.Buildings.Shell.Water heating
pyCIMS.Canada.Alberta.Residential.Buildings.Dishwashing
pyCIMS.Canada.Alberta.Residential.Buildings.Clothes washing
pyCIMS.Canada.Alberta.Electricity
pyCIMS.Canada.Alberta.Electricity.Generation
pyCIMS.Canada.Alberta.Electricity.Generation.Base load
pyCIMS.Canada.Alberta.Electricity.Generation.Base load.Conventional
pyCIMS.Canada.Alberta.Electricity.Generation.Base load.Renewables
pyCIMS.Canada.Alberta.Electricity.Transmission Losses
pyCIMS.Canada.Alberta.Natural Gas
pyCIMS.Canada.Alberta.Solar
pyCIMS.Canada.Alberta.Wind
pyCIMS.Canada.Alberta.Water


In [4]:
g = my_model.graph

In [59]:
def get_subgraph(graph, node_types):
    nodes = [n for n, a in graph.nodes(data=True) if a['type'] in node_types]
    sub_g = graph.subgraph(nodes).copy()
    return sub_g

In [62]:
g_d = get_subgraph(g, ["demand", "standard"])

In [64]:
g_s = get_subgraph(g, "supply")  

In [5]:
def stack_trace(name):
    exc_type, exc_value, exc_traceback = sys.exc_info()
    print(f'-> Service requested failed for node {name} <- .\n\n ** Printing stack traceback:\n')
    traceback.print_exception(exc_type, exc_value, exc_traceback,
                               limit=2, file=sys.stdout)   

In [6]:
years = my_model.years
fuels = my_model.fuels
fuels = ['.'.join(fuels[i].split('.')[-1:]) for i, val in enumerate(fuels)]

In [92]:
class Node(ABC):
    __metaclass__  = ABCMeta
    
    def __init__(self, subdict, cur_year='2000'):
        self.subdict = subdict
        self.cur_year = cur_year
        name_sg = subdict[cur_year]['Service provided']
        requested_sg = subdict[cur_year]['Service requested']
        
        # name of current node/ service provided and competition type: common to all nodes
        self.node_name = list(name_sg.keys())[0]
        # name of service requested
        self.requested = list(requested_sg.keys())[0]   
        self.request_val = None
        
        try:
            self.node_type = subdict['competition_type']
        except:
            stack_trace(self.node_name)
            

    @abstractmethod
    def get_request(self):
        if self.node_type == "root" or self.node_type == "country":
            pass
        
        else:
            request_name = self.requested
            self.request_val = self.subdict[self.cur_year]['Service requested'][request_name]['year_value']
            self.isnum = isinstance(self.request_val, (int, float, complex)) and not isinstance(self.request_val, bool) 
            
            if self.node_type == "region": 
                self.population = self.subdict[self.cur_yearyear]["Attribute"]["Population"]["year_value"]
                
                if isnum:
                    request = request_val * self.population
                else:
                    print(f"Non numerical value for service requested in node {self.node_name}")
                    request = self.population
                    
                return request    
            else:
                if isnum:
                    print("fixed node")
                    return request_val
                else:
                    print(f"Non numerical value for service requested in node {self.node_name}")
                    return 1
            
        

In [99]:
class Root(Node):
    def __init__(self, subdict):
        super().__init__(subdict)
        
        self.node_type = "root"

        print(self.get_request())
        
    def get_request(self):
        super().get_request()
        return f"root"

In [100]:
class Country(Node):
    def __init__(self, subdict):
        super().__init__(subdict)
        self.node_type = "country"

        print(self.get_request())
        
    def get_request(self):
        return f"country"        
    

In [101]:
class Region(Node):
    def __init__(self, subdict):
        super().__init__(subdict)
        
        self.node_type = "region"
        print(self.get_request())
        
    def get_request(self):
        return f"country"           
        

In [102]:
class FixedRatio(Node):
    def __init__(self, subdict):
        super().__init__(subdict)
        try:
            self.provides = subdict[self.cur_year]['Service provided'][self.node_name]['year_value']
        except:
            stack_trace(self.node_name)     
        print(self.get_request())
        
    def get_request(self):
        super().get_request()
        return f"fixed"    
            

In [106]:
class TechCompete(Node):
    '''All the calculations are here'''
    def __init__(self, subdict):
        super().__init__(subdict)

        try:
            self.provides = subdict[self.cur_year]['Service provided'][self.node_name]['year_value']
        except:
            stack_trace(self.node_name)     
            
        print(self.get_request())
        
    def get_request(self):
        super().get_request()
        return f"compete"    
    
    def get_demand(self):
        pass

In [107]:
class FixedMarketShare(Node):
    def __init__(self, subdict):
        super().__init__(subdict)

        try:
            self.provides = subdict[self.cur_year]['Service provided'][self.node_name]['year_value']
        except:
            stack_trace(self.node_name)     
            
        print(self.get_request())
        
    def get_request(self):
        super().get_request()
        return f"fixed marketshare"    

In [108]:
class SupplyNoTech(Node):
    def __init__(self, subdict):
        super().__init__(subdict)
    
        print(self.get_request())
        
    def get_request(self):
        super().get_request()
        return f"supply no tech"  
    
    def no_price_available(self):
        if self.subdict[]

In [69]:
galb = g.nodes[search_nodes( g, "Alberta")[0]]

In [103]:
FixedRatio(galb)

fixed node
fixed


In [16]:
f = FixedRatio(g.node[search_nodes( g, "Alberta")[0]])

In [21]:
print(f.requested)

Canada


In [25]:
def traverse_graph(sub_graph, node_process_func):
    # Find the root of the sub-graph
    root = [n for n, d in sub_graph.in_degree() if d == 0][0]

    # Find the distance from the root to each node in the sub-graph
    dist_from_root = nx.single_source_shortest_path_length(sub_graph, root)

    # Start the traversal
    sg_cur = copy.deepcopy(sub_graph)
    visited = []

    while len(sg_cur.nodes) > 0:
        active_front = [n for n, d in sg_cur.in_degree if d == 0]

        if len(active_front) > 0:
            # Choose a node on the active front
            n_cur = active_front[0]
            # Process that node in the sub-graph
            node_process_func(sub_graph, n_cur)
        else:
            # Resolve a loop
            candidates = {n: dist_from_root[n] for n in sg_cur}
            n_cur = min(candidates, key=lambda x: candidates[x])
            # Process chosen node in the sub-graph, using estimated values from their parents
            node_process_func(sub_graph, n_cur, with_estimates=True)

        visited.append(n_cur)
        sg_cur.remove_node(n_cur)

In [26]:
def get_subgraph(g, node_types):
    nodes = [n for n, a in g.nodes(data=True) if a['type'] in node_types]
    sub_g = g.subgraph(nodes).copy()
    return sub_g

In [35]:
# def calc_demand(prices):
#     def demand_process_func(g, node, fuels, with_estimates=False):
#         g.nodes[node]['demand'] = {f: random.randint(10, 50) for f in fuels}

In [37]:
def demand_process_func(g, node, with_estimates=False):
    g.nodes[node]['demand'] = {f: random.randint(10, 50) for f in fuels}

In [38]:
def calculate_service_cost(sub_graph):
    def calculate_node_sc(node):
        children = sub_graph[node]
        if len(children) == 0:
            service_cost = 10
        else:
            service_cost = sum([calculate_node_sc(c) for c in children])
        sub_graph.nodes[node]['service cost'] = service_cost
        return service_cost

    roots = [n for n, d in sub_graph.in_degree() if d == 0]
    for root in roots:
        print(calculate_node_sc(root))

In [29]:
g.nodes['pyCIMS.Canada.Alberta.Residential.Buildings']['2000']

{'Service provided': {'Buildings': {'source': None,
   'branch': 'pyCIMS.Canada.Alberta.Residential.Buildings',
   'unit': 'building',
   'year_value': None}},
 'technologies': {'Single Family Detached': {'Market share': {'source': None,
    'branch': None,
    'unit': '%',
    'year_value': 0.719409242},
   'Service requested': None},
  'Single Family Attached': {'Market share': {'source': None,
    'branch': None,
    'unit': '%',
    'year_value': 0.1},
   'Service requested': None},
  'Apartment': {'Market share': {'source': None,
    'branch': None,
    'unit': '%',
    'year_value': 0.14},
   'Service requested': None},
  'Mobile': {'Market share': {'source': None,
    'branch': None,
    'unit': '%',
    'year_value': 0.040590758000000005},
   'Service requested': [{'source': None,
     'branch': 'pyCIMS.Canada.Alberta.Residential.Buildings.Shell',
     'unit': 'm2 floorspace/building',
     'year_value': 91.72141404661348},
    {'source': None,
     'branch': 'pyCIMS.Canada.Alb

In [ ]:
class Node:
    def __init__(self, subdict):
        

In [52]:
g.nodes()

NodeDataView({'pyCIMS': None, 'pyCIMS.Canada': None, 'pyCIMS.Canada.Alberta': None, 'pyCIMS.Canada.Alberta.Residential': None, 'pyCIMS.Canada.Alberta.Residential.Buildings': None, 'pyCIMS.Canada.Alberta.Residential.Buildings.Shell': None, 'pyCIMS.Canada.Alberta.Residential.Buildings.Shell.Space heating': None, 'pyCIMS.Canada.Alberta.Residential.Buildings.Shell.Space heating.Furnace': None, 'pyCIMS.Canada.Alberta.Residential.Buildings.Shell.Water heating': None, 'pyCIMS.Canada.Alberta.Residential.Buildings.Dishwashing': None, 'pyCIMS.Canada.Alberta.Residential.Buildings.Clothes washing': None, 'pyCIMS.Canada.Alberta.Electricity': None, 'pyCIMS.Canada.Alberta.Electricity.Generation': None, 'pyCIMS.Canada.Alberta.Electricity.Generation.Base load': None, 'pyCIMS.Canada.Alberta.Electricity.Generation.Base load.Conventional': None, 'pyCIMS.Canada.Alberta.Electricity.Generation.Base load.Renewables': None, 'pyCIMS.Canada.Alberta.Electricity.Transmission Losses': None, 'pyCIMS.Canada.Alberta.N

In [49]:
alberta = search_nodes( g, "Buildings")[0]
g.node[alberta]#['2000']

{'type': 'demand',
 'competition_type': 'fixed market shares',
 '2000': {'Service provided': {'Buildings': {'source': None,
    'branch': 'pyCIMS.Canada.Alberta.Residential.Buildings',
    'unit': 'building',
    'year_value': None}},
  'technologies': {'Single Family Detached': {'Market share': {'source': None,
     'branch': None,
     'unit': '%',
     'year_value': 0.719409242},
    'Service requested': None},
   'Single Family Attached': {'Market share': {'source': None,
     'branch': None,
     'unit': '%',
     'year_value': 0.1},
    'Service requested': None},
   'Apartment': {'Market share': {'source': None,
     'branch': None,
     'unit': '%',
     'year_value': 0.14},
    'Service requested': None},
   'Mobile': {'Market share': {'source': None,
     'branch': None,
     'unit': '%',
     'year_value': 0.040590758000000005},
    'Service requested': [{'source': None,
      'branch': 'pyCIMS.Canada.Alberta.Residential.Buildings.Shell',
      'unit': 'm2 floorspace/buildin

In [39]:
        
equilibrium_threshold=0.05

for year in years:

    # Find the demand sub-graph
    g_demand = get_subgraph(g, ['demand', 'standard'])
    
    # Traverse the sub-graph, processing as we encounter nodes
    traverse_graph(g_demand, demand_process_func)    

TypeError: demand_process_func() missing 1 required positional argument: 'fuels'

In [30]:
                # Calculate Service Cost for every node in the sub-graph
                # calculate_service_cost(g_demand)

                # Traverse the sub-graph, processing as we encounter nodes
                traverse_graph(g_demand, demand_process_func)

                # Aggregate demand
                demand_by_fuel = self.aggregate(g_demand, ['demand'], agg_func=sum)
                return demand_by_fuel

            def calc_supply(demand):
                def supply_process_func(node, with_estimates=False):
                    pass

                # Find the supply sub-graph
                g_supply = get_subgraph(['supply', 'standard'])

                # Traverse the sub-graph processing as we encounter nodes
                traverse_graph(g_supply, supply_process_func)

                # Find the prices by fuel (Note: Unsure. This might actually happen in the previous step...)
                fuel_prices = {}

                return fuel_prices

            def equilibrium_check(dict1, dict2, threshold):
                for fuel in dict1:
                    abs_diff = abs(dict1[fuel] - dict2[fuel])
                    rel_diff = abs_diff / dict1[fuel]
                    if rel_diff > threshold:
                        return False

                return True

            prev_prices = {}
            equilibrium = False
            while not equilibrium:
                curr_demand = calc_demand(prev_prices)
                curr_prices = calc_supply(curr_demand)
                # equilibrium = equilibrium_check(prev_prices, curr_prices, equilibrium_threshold)
                equilibrium = True

                # prev_prices = curr_prices

            # TODO: Add finishing procedures. Ex. Storing resulting prices and demands

        for y in self.years:
            run_year(y)

    def search_nodes(self, search_term):
        """Search nodes to see if there is one that contains the search term in the final component of its name"""
        def search(name):
            components = name.split('.')
            last_comp = components[-1]
            return search_term.lower() in last_comp.lower()

        return [n for n in self.graph.nodes if search(n)]

    def aggregate(self, sub_graph, agg_key, agg_func=sum):
        """
        Sum agg_val across all nodes in a given subgraph.
        :param sub_graph: nx.Graph. The sub-graph to be aggregated over.
        :param agg_key: List of str. The key list needed to access the values to be aggregated. Will be used to
        :param agg_func:
        :return:
        """

        def get_val(dict, key_list, name_TEMP):
            value = dict
            for k in key_list:
                value = value[k]
            return value

        values_by_node = [get_val(data, agg_key, name) for name, data in sub_graph.nodes(data=True)]

        all_values = [(k, v) for values in values_by_node for k, v in values.items()]

        # Make value lists, separated by key
        value_lists = {}
        for k, v in all_values:
            try:
                value_lists[k].append(v)
            except KeyError:
                value_lists[k] = [v]

        # Aggregate each list by applying agg_function
        aggregates = {k: agg_func(v) for k, v in value_lists.items()}

        return aggregates

IndentationError: unexpected indent (<ipython-input-30-97287de04faf>, line 13)

In [ ]:

# def life_cycle_cost(data):
#     """Note: operating cost is always empty"""
#     if data["Capital cost"]["year_value"] is not None:
#         capcost = data["Capital cost"]["year_value"]
#     else:
#         capcost = 0.0
#     if data["Operating cost"]["year_value"] is not None:
#         opcost = data["Operating cost"]["year_value"]
#     else:
#         opcost = 0.0
        
#     lcc = capcost + opcost 
#     return lcc
    
# def market_share(alltechs, v=10):
#     numtechs = len(alltechs)
#     all_lccs  = {}
#     marketshares = {}
    
#     # alltechs is a dict of all (directly) competing tech subgraphs
#     for tech, subgraph in alltechs.items():
#         all_lccs[tech] = life_cycle_cost(subgraph)
#     sum_v = sum(lcc**(-1.0*v) for lcc in all_lccs.values() if lcc > 0.0)
    
#     for tech, val in all_lccs.items():
#         if val > 0.0:
#             marketshares[tech] = (val**(-1.0*v))/sum_v
#         else:
#             marketshares[tech] = 0.0
        
#     return marketshares    

In [ ]:
alberta = search_nodes( g_d, "Alberta")[0]
g_d.node[alberta]['2000'].keys()

In [ ]:
# get start values

population = g_d.node[alberta]['2000']["Attribute"]["Population"]["year_value"]
gdp = g_d.node[alberta]['2000']["Attribute"]["GDP"]["year_value"]
ngas_price = g_d.node[alberta]['2000']["Price"]["Natural Gas"]["year_value"]
electricity_price = g_d.node[alberta]['2000']["Price"]["Electricity"]["year_value"]

household_per_person = g_d.node[alberta]['2000']["Service requested"]["Residential"]["year_value"]
num_households = household_per_person * population

In [ ]:
residential = search_nodes( g_d, "Residential")[0]
g_d.node[residential]['2000'].keys()

In [ ]:
ngas_multiplier = g_d.node[residential]['2000']["Price Multiplier"]["Natural Gas"]["year_value"]
electricity_multiplier = g_d.node[residential]['2000']["Price Multiplier"]["Electricity"]["year_value"]

electricity_price *= electricity_multiplier
ngas_price *= ngas_multiplier

In [ ]:
# Start Buildings (parent of 1st tech compete)

buildings = search_nodes( g_d, "Buildings")[0]
g_d.node[buildings]['2000']['technologies'].keys()

In [ ]:
marketshare_famdet = g_d.node[buildings]['2000']['technologies']['Single Family Detached']['Market share']['year_value']
marketshare_famatt = g_d.node[buildings]['2000']['technologies']['Single Family Attached']['Market share']['year_value']
marketshare_apartment = g_d.node[buildings]['2000']['technologies']['Aprtment']['Market share']['year_value']
marketshare_mobile = g_d.node[buildings]['2000']['technologies']['Mobile']['Market share']['year_value']

In [ ]:
g_d.node[buildings]['2000']['technologies']#['Single Family Detached']#['Service requested']#['Service requested'].keys()

In [ ]:
# PROBLEM: Dict gets rewritten everytime the key is the same, so we can't get any values before clothes washing (eg for shell)

# g_d.node[buildings]['2000']['Service requested']

In [ ]:
# place values by hand due to problem above

space_per_famdet = 138.52 # g_d.node[buildings]['2000']['Service requested']['Single Family Detached']
space_per_famatt = 107.37
space_per_apartment = 84.76
space_per_mobile = 91.72

all_famdet = space_per_famdet*(marketshare_famdet * num_households)
all_famatt = space_per_famatt*(marketshare_famatt * num_households)
all_apartment = space_per_apartment*(marketshare_apartment * num_households)
all_mobile = space_per_mobile*(marketshare_mobile * num_households)

all_space = all_famdet + all_famatt + all_apartment + all_mobile

In [ ]:
# shell

shell = search_nodes( g_d, "Shell")[0]
g_d.node[shell]['2000']['technologies'].keys()

In [ ]:
pre1960_sg = g_d.node[shell]['2000']['technologies']['Pre-1960']
post1960_sg = g_d.node[shell]['2000']['technologies']['Post-1960']
standard_sg = g_d.node[shell]['2000']['technologies']['Standard']
R2000_sg = g_d.node[shell]['2000']['technologies']['R2000']
LEED_sg = g_d.node[shell]['2000']['technologies']['LEED']


# fetch v here
v_shell = g_d.node[shell]['2000']['Heterogeneity']['v']['year_value']

compete_shell = {"Pre-1960": pre1960_sg, 
                 "Post-1960": post1960_sg,
                 "Stadard": standard_sg,
                 "R2000": R2000_sg,
                 "LEED": LEED_sg}

In [ ]:
marketshare_pre1960 = pre1960_sg["Market share"]["year_value"]
marketshare_post1960 = post1960_sg["Market share"]["year_value"]
marketshare_standard = standard_sg["Market share"]["year_value"]
marketshare_R2000 = R2000_sg["Market share"]["year_value"]
marketshare_LEED = LEED_sg["Market share"]["year_value"]


In [ ]:
g_d.node[shell]['2000']['technologies'].keys()

In [ ]:
# plug into 'output':is this correct

g_d.node[shell]['2000']['technologies']['Pre-1960']["Output"]["year_value"] = marketshare_pre1960*all_space
g_d.node[shell]['2000']['technologies']['Post-1960']["Output"]["year_value"] = marketshare_post1960*all_space
g_d.node[shell]['2000']['technologies']['Standard']["Output"]["year_value"] = marketshare_standard*all_space
g_d.node[shell]['2000']['technologies']['R2000']["Output"]["year_value"] = marketshare_R2000*all_space
g_d.node[shell]['2000']['technologies']['LEED']["Output"]["year_value"] = marketshare_LEED*all_space

# # here same as output: just multiplied by 1
pre1960 = g_d.node[shell]['2000']['technologies']['Pre-1960']['Service requested']["year_value"]*marketshare_pre1960*all_space
post1960 = g_d.node[shell]['2000']['technologies']['Post-1960']['Service requested']["year_value"]*marketshare_post1960*all_space
standard = g_d.node[shell]['2000']['technologies']['Standard']['Service requested']["year_value"]*marketshare_standard*all_space
R2000 = g_d.node[shell]['2000']['technologies']['R2000']['Service requested']["year_value"]*marketshare_R2000*all_space
LEED = g_d.node[shell]['2000']['technologies']['LEED']['Service requested']["year_value"]*marketshare_LEED*all_space

shellGJ = pre1960 + post1960 + standard + R2000 + LEED
# note: in 2000, only pre- and post-1960 are available

In [ ]:
#Space Heating

space_heating = search_nodes(g_d, 'space heating')[0]
g_d.node[space_heating]['2000']['technologies'].keys()

In [ ]:
electric_baseboard_sg = g_d.node[space_heating]['2000']['technologies']['Electric baseboard']
furnace_sg = g_d.node[space_heating]['2000']['technologies']['Furnace']
# fetch v here
v_space_heating = g_d.node[space_heating]['2000']['Heterogeneity']['v']['year_value']

compete_space_heating = {"Electric baseboard": electric_baseboard_sg, 
                         "Furnace": furnace_sg}

In [ ]:
marketshare_electric_baseboard = electric_baseboard_sg["Market share"]["year_value"]
marketshare_furnace = furnace_sg["Market share"]["year_value"]

In [ ]:
electric_baseboard_sg.keys()

In [ ]:
g_d.node[space_heating]['2000']['technologies']['Electric baseboard']["Output"]["year_value"] = marketshare_electric_baseboard*shellGJ
g_d.node[space_heating]['2000']['technologies']['Furnace']["Output"]["year_value"] = marketshare_furnace*shellGJ

In [ ]:
electric_baseboard_requested = g_d.node[space_heating]['2000']['technologies']['Electric baseboard']["Service requested"]["year_value"]*marketshare_electric_baseboard*shellGJ   
furnace_requested = g_d.node[space_heating]['2000']['technologies']['Furnace']["Service requested"]["year_value"]*marketshare_furnace*shellGJ   

In [ ]:
# Furnace (sum with baseboard after)

furnace = search_nodes( g_d, "Furnace")[0]
g_d.node[furnace]['2000']['technologies'].keys()

In [ ]:
natural_gas_sg = g_d.node[furnace]['2000']['technologies']['Natural Gas_existing']
pump_air_sg = g_d.node[furnace]['2000']['technologies']['Heat pump_air source']
pump_ground_sg = g_d.node[furnace]['2000']['technologies']['Heat pump_ground source']
natural_gas_higheff_sg = g_d.node[furnace]['2000']['technologies']['Natural Gas_high efficiency']
# fetch v here
v_furnace = g_d.node[furnace]['2000']['Heterogeneity']['v']['year_value']

compete_furnace = {"Natual Gas": natural_gas_sg, 
                   "Pump Air": pump_air_sg,
                   "Pump Ground": pump_ground_sg,
                   "Natural Gas High Efficiency": natural_gas_higheff_sg}

In [ ]:
marketshare_natural_gas = natural_gas_sg["Market share"]["year_value"]
marketshare_pump_air = pump_air_sg["Market share"]["year_value"]
marketshare_pump_ground = pump_ground_sg["Market share"]["year_value"]
marketshare_natural_gas_higheff = natural_gas_higheff_sg["Market share"]["year_value"]


In [ ]:
g_d.node[furnace]['2000']['technologies']['Natural Gas_existing']["Output"]["year_value"] = marketshare_natural_gas*furnace_requested
g_d.node[furnace]['2000']['technologies']['Heat pump_air source']["Output"]["year_value"] = marketshare_pump_air*furnace_requested 
g_d.node[furnace]['2000']['technologies']['Heat pump_ground source']["Output"]["year_value"] = marketshare_pump_ground*furnace_requested 
g_d.node[furnace]['2000']['technologies']['Natural Gas_high efficiency']["Output"]["year_value"] = marketshare_natural_gas_higheff*furnace_requested 

In [ ]:
natural_gas_requested = g_d.node[furnace]['2000']['technologies']['Natural Gas_existing']["Service requested"]["year_value"]*marketshare_natural_gas*furnace_requested   
natural_gas_requested += g_d.node[furnace]['2000']['technologies']['Natural Gas_high efficiency']["Service requested"]["year_value"]*marketshare_natural_gas_higheff*furnace_requested   

In [ ]:
electricity_requested = g_d.node[furnace]['2000']['technologies']['Heat pump_air source']["Service requested"]["year_value"]*marketshare_pump_air*furnace_requested   
electricity_requested += g_d.node[furnace]['2000']['technologies']['Heat pump_ground source']["Service requested"]["year_value"]*marketshare_pump_ground*furnace_requested   

In [ ]:
electricity_requested += electric_baseboard_requested 

In [ ]:
#Supply: electricity

electricity = search_nodes( g, "Electricity")[0]
g.node[electricity]['2000'].keys()

In [ ]:
g[electricity]

In [ ]:
space_heating = search_nodes(g, 'space heating')[0]
furnace = search_nodes(g, 'Furnace')[0]

In [ ]:
g.node[space_heating]['2000']['technologies'].keys()

In [ ]:
electric_baseboard_sg = g.node[space_heating]['2000']['technologies']['Electric baseboard']
furnace_sg = g.node[space_heating]['2000']['technologies']['Furnace']
# fetch v here
v_space_heating = g.node[space_heating]['2000']['Heterogeneity']['v']['year_value']

compete_space_heating = {"Electric baseboard": electric_baseboard_sg, 
                         "Furnace": furnace_sg}

In [ ]:
g.node[furnace]['2000']['technologies'].keys()

In [ ]:
# heat 

natural_gas_existing_sg = g.node[furnace]['2000']['technologies']['Natural Gas_existing']
heat_pump_air_source_sg = g.node[furnace]['2000']['technologies']['Heat pump_air source'] 
# heat_pump_ground_source_sg = g.node[furnace]['2000']['technologies']['Heat pump_ground source'] # unavail
natural_gas_high_efficiency_sg = g.node[furnace]['2000']['technologies']['Natural Gas_high efficiency']
v_furnace = g.node[furnace]['2000']['Heterogeneity']['v']['year_value']
compete_furnace = {"Natural Gas_existing": natural_gas_existing_sg,
                   "Heat pump_air source": heat_pump_air_source_sg,
                   "Natural Gas_high efficiency": natural_gas_high_efficiency_sg}

In [ ]:
electricity = search_nodes(g_supply, 'electricity')[0]
natural_gas = search_nodes(g_supply, 'Natural Gas')[0]

In [ ]:
electricity_sg = g_supply.node[electricity]['2000']
natural_gas_sg = g_supply.node[natural_gas]['2000']

In [ ]:
# initialize service cost (2000 weighted LLC * MS)

In [ ]:
# get initial values
natural_gas_existing_sg
# natural_gas_high_efficiency_sg




In [ ]:
a = market_share(compete_furnace)

In [ ]:
a

In [ ]:
class Demand:
    '''
    
    - **kwargs: dict of named arguments in "argname": value pairs to specify used economic indicators (GDP, population, etc)
    '''

    def __init__(self, data, v=10, keyfuel="kind", return_all_info=False, **kwargs):
        self.keyfuel = keyfuel
        
        self.dtechs = data["demand_step"]
        self.dfuels = data["supply_step"]
        
        lccs = self.lcc_over_tech()
        
        if return_all_info:
            tech_demand, tech_mshares = self.quantity_demanded(lccs, v=v, return_all_info=True)
            fuel_demand = self.demand_by_fuel(tech_demand)
            self.fuel_demand, self.tech_demand, self.tech_mshares = fuel_demand, tech_demand, tech_mshares
        else:
            tech_demand = self.quantity_demanded(lccs, v=v, return_all_info=False)
            fuel_demand = self.demand_by_fuel(tech_demand)
            self.fuel_demand = fuel_demand
        
        

        
    def get_quantity(self, ms, fuel_quantity):
        demand = ms * fuel_quantity
        return demand
    
    def lcc_over_tech(self, keytech = "kind", keysubtech = "subtypes",
                            keyprice = "price", keycapcost = "cap_cost"):
        # TODO: use some kwargs in lcc method in case there are arbitrary numbers of params
        lccs = {}
        # techdict is the entire subtree for the kth competing tech (demand side)
        for techdict in self.dtechs:
            # get the tech name (key) to use as a label for lcc
            tech_k = techdict[keytech]
            lccs[tech_k] = {}
            # fetch all attributes/values for the (sub)tech that demands a fuel
            # by `subtech` I mean e.g.: tech: furnace, subtech: type of furnace
            subtechs = techdict[keysubtech]            
            # loop over `subtechs`, get subtech name (key) and vals (all attributes associated w/ subtech)
            for subtech, vals in subtechs.items():
                # unpack components needed for lcc
                price = vals[keyprice]
                cc = vals[keycapcost]
                lccs[tech_k][subtech] = self.life_cycle_cost(last_price=price, cap_cost=cc) # kwargs here
        return lccs        
                
                
    def quantity_demanded(self, lccs, v=10, return_all_info=True, keytech="kind", 
                                keysubtech="subtypes", keyquant="total_demand"):
        mshares, quants = {}, {}

        # techdict is the entire subtree for the kth competing tech (demand side)            
        for techdict in self.dtechs:
            # fetch all attributes/values for the (sub)tech that demands a fuel
            # by `subtech` I mean e.g.: tech: furnace, subtech: type of furnace
            subtechs = techdict[keysubtech]
            # loop over `subtechs`, get subtech name (key) and vals (all attributes associated w/ subtech)                
                
            for tech, vals in subtechs.items():
                mshares[tech] = self.market_share(lccs[techdict[keytech]], tech, v=v)
                quants[tech] = self.get_quantity(mshares[tech], techdict[keyquant])     
                    
        if return_all_info:
            return quants, mshares
        else:
            return quants
        
    def demand_by_fuel(self, quants, keytechfuel="fuel", keysubtech="subtypes"):
        by_fuel, sum_quants = {}, {}

        # get only numbers for a given fuel, in both techs
        for fkind in self.dfuels:
            cur_fuel = fkind[self.keyfuel]
            by_fuel[cur_fuel] = filter_obs(data, fuel_type=cur_fuel, val="fuel", verbose=False)
            sum_quants[cur_fuel] = sum(quants[subtype] for subtype in by_fuel[cur_fuel].keys())
        return sum_quants    
                    



#### Define classes for Supply and Demand:

In [ ]:
class Demand:
    '''
    
    - **kwargs: dict of named arguments in "argname": value pairs to specify used economic indicators (GDP, population, etc)
    '''

    def __init__(self, data, v=10, keyfuel="kind", return_all_info=False, **kwargs):
        self.keyfuel = keyfuel
        
        self.dtechs = data["demand_step"]
        self.dfuels = data["supply_step"]
        
        lccs = self.lcc_over_tech()
        
        if return_all_info:
            tech_demand, tech_mshares = self.quantity_demanded(lccs, v=v, return_all_info=True)
            fuel_demand = self.demand_by_fuel(tech_demand)
            self.fuel_demand, self.tech_demand, self.tech_mshares = fuel_demand, tech_demand, tech_mshares
        else:
            tech_demand = self.quantity_demanded(lccs, v=v, return_all_info=False)
            fuel_demand = self.demand_by_fuel(tech_demand)
            self.fuel_demand = fuel_demand
        
        
    ## Methods for demand side calculations 
    def life_cycle_cost(self, last_price, cap_cost=0):
        lcc = (last_price + cap_cost)
        return lcc
    
    def market_share(self, lcc, techname, v=10):
        # lcc is a dict
        sum_v = sum(lcc[item]**(-1.0*v) for item in lcc)
        ms = (lcc[techname]**(-1.0*v))/sum_v
        return ms
        
    def get_quantity(self, ms, fuel_quantity):
        demand = ms * fuel_quantity
        return demand
    
    def lcc_over_tech(self, keytech = "kind", keysubtech = "subtypes",
                            keyprice = "price", keycapcost = "cap_cost"):
        # TODO: use some kwargs in lcc method in case there are arbitrary numbers of params
        lccs = {}
        # techdict is the entire subtree for the kth competing tech (demand side)
        for techdict in self.dtechs:
            # get the tech name (key) to use as a label for lcc
            tech_k = techdict[keytech]
            lccs[tech_k] = {}
            # fetch all attributes/values for the (sub)tech that demands a fuel
            # by `subtech` I mean e.g.: tech: furnace, subtech: type of furnace
            subtechs = techdict[keysubtech]            
            # loop over `subtechs`, get subtech name (key) and vals (all attributes associated w/ subtech)
            for subtech, vals in subtechs.items():
                # unpack components needed for lcc
                price = vals[keyprice]
                cc = vals[keycapcost]
                lccs[tech_k][subtech] = self.life_cycle_cost(last_price=price, cap_cost=cc) # kwargs here
        return lccs        
                
                
    def quantity_demanded(self, lccs, v=10, return_all_info=True, keytech="kind", 
                                keysubtech="subtypes", keyquant="total_demand"):
        mshares, quants = {}, {}

        # techdict is the entire subtree for the kth competing tech (demand side)            
        for techdict in self.dtechs:
            # fetch all attributes/values for the (sub)tech that demands a fuel
            # by `subtech` I mean e.g.: tech: furnace, subtech: type of furnace
            subtechs = techdict[keysubtech]
            # loop over `subtechs`, get subtech name (key) and vals (all attributes associated w/ subtech)                
                
            for tech, vals in subtechs.items():
                mshares[tech] = self.market_share(lccs[techdict[keytech]], tech, v=v)
                quants[tech] = self.get_quantity(mshares[tech], techdict[keyquant])     
                    
        if return_all_info:
            return quants, mshares
        else:
            return quants
        
    def demand_by_fuel(self, quants, keytechfuel="fuel", keysubtech="subtypes"):
        by_fuel, sum_quants = {}, {}

        # get only numbers for a given fuel, in both techs
        for fkind in self.dfuels:
            cur_fuel = fkind[self.keyfuel]
            by_fuel[cur_fuel] = filter_obs(data, fuel_type=cur_fuel, val="fuel", verbose=False)
            sum_quants[cur_fuel] = sum(quants[subtype] for subtype in by_fuel[cur_fuel].keys())
        return sum_quants    
                    



In [ ]:
class Supply:
    def __init__(self, data, v=10, keyfuel="kind", return_all_info=True, **kwargs):
        self.keyfuel = keyfuel
        
        self.sfuels = data["supply_step"]
        lccs = self.lcc_over_production()
        
        if return_all_info:
            self.tot_price, self.price, self.sup_mshares = self.get_fuelprice(lccs, v)
        else:
            self.tot_price = self.get_fuelprice(lccs, v)            
            
            
    ## Methods for calculating demand side 
    def life_cycle_cost(self, last_price, cap_cost=0):
        lcc = (last_price + cap_cost)
        return lcc
        
    def market_share(self, lcc, prodname, v=10):
        # lcc is a dict
        sum_v = sum(lcc[item]**(-1.0*v) for item in lcc)
        ms = (lcc[prodname]**(-1.0*v))/sum_v
        return ms
        
    def get_price(self, ms, lcc):
        price = ms*lcc       
        return price
                
        
        
    def lcc_over_production(self, keyprod = "subtypes", keyprice = "price", keycapcost = "cap_cost"):  
        lccs = {}        
        for fueldict in self.sfuels:
            # loop through each fuel, fueldict is entire subtree below each type of fuel
            fueltype = fueldict[self.keyfuel]
            lccs[fueltype] = {}
            
            for prod, vals in fueldict[keyprod].items():
                # loop through each production type (say 'hydro' if fuel is 'electricity')
                lccs[fueltype][prod] = self.life_cycle_cost(last_price=vals[keyprice], cap_cost=vals[keycapcost])
        return lccs    
    
    def get_fuelprice(self, lccs, v = 10, keyprod = "subtypes", keyprice = "price", 
                            keycapcost = "cap_cost", return_all_info = True): 
        # distinguish the 2 types of lccs (one for added, one for price)
        lcc_model, mshares, price, tot_price = {}, {}, {}, {}
        
        for fueldict in self.sfuels:
            # loop through each fuel, fueldict is entire subtree below each type of fuel
            fueltype = fueldict[self.keyfuel]                          
            lcc_model[fueltype], mshares[fueltype], price[fueltype] = {}, {}, {}
            for prod, vals in fueldict[keyprod].items():               
                lcc_model[fueltype][prod] = self.life_cycle_cost(last_price=vals[keyprice], cap_cost=vals[keycapcost])
                mshares[fueltype][prod] = self.market_share(lccs[fueltype], prod, v=v)                
                price[fueltype][prod] = self.get_price(mshares[fueltype][prod], lcc_model[fueltype][prod])    # (total_quants["electricity"])   
            tot_price[fueltype] = sum(price[fueltype][p]/len(fueldict[keyprod].keys()) for p in fueldict[keyprod].keys())
            # TODO!! check averaging method
            
        if return_all_info:
            return tot_price, price, mshares 
        else:
            return tot_price
            
            

In [ ]:
class SupplyDemand:
    '''
    SUPPLY-DEMAND MODEL:
        - Demand: estimate Quantity
        - Supply: estimate Price
        - Repeat using updated prices    
    '''    
    def __init__(self, data, verbose=True,
                             demandkey="demand_step",
                             supplykey="supply_step",
                             reskeys=("dem_price_by_fuel", "quant_by_fuel", "sup_price_by_fuel"),
                             infokeys=("fuels", "price_diff"),
                             keyfuel="kind"):
        self.data = data   
        self.techs = data[demandkey]
        self.fuels = data[supplykey]
        self.tempdata = deepcopy(data) # check in case data is unsafe to change
        self.keyfuel = keyfuel
        self.demandkey = demandkey
        self.supplykey = supplykey
        
        # may want to make keys editable from args instead
        self.reskeys = reskeys
        self.results = DictAppend()
        self.infokeys = infokeys
        self.info = DictAppend()
        
        self.setup()
        
    def setup(self):
        
        self.demand = Demand(self.data)
        quant_hat = self.demand.fuel_demand       

        init_price = {ft[self.keyfuel]: find_price_for_fuel( fuel_type = ft[self.keyfuel], data = self.techs ) for ft in self.fuels}
        self.supply = Supply(self.data)
        price_hat = self.supply.tot_price
        new_res = [init_price, quant_hat, price_hat]

        for num, res in enumerate(new_res):
            for resname, resval in res.items():
                self.results[self.reskeys[num]][resname].append(resval)
                
                
                
    def iterate(self, relative_tol=0.05, max_iter=10, n_iter=0):
        price_diff, price_thres, ge_thres = {}, {}, {}
        
        print(self.fuels)
        
        for ft in self.fuels: 
            # storing fuels used in current competition cluster
            f = ft[self.keyfuel]
            self.info[self.infokeys[0]].append(ft[self.keyfuel]) 

            # last demand price recoreded for fuel f (here just initial demand price)
            last_price = self.results[self.reskeys[0]][f][-1] 
            # last supply price recoreded for fuel f
            cur_price = self.results[self.reskeys[2]][f][-1]
            price_diff[f] = np.abs(last_price - cur_price)
            price_thres[f] = cur_price * relative_tol
            ge_thres[f] = price_diff[f] > price_thres[f]
            # store price difference
            self.info[self.infokeys[1]].append( price_diff )
        
        while any(ge_thres) and n_iter <= max_iter:
            # iterate until price diff >= relative_thres and until max number of iters reached
            for fuel, is_ge in ge_thres.items():
                new_price = self.results[self.reskeys[2]][fuel][-1]
                # change price for fuel belonging to all techs in initial `dataset`
                change_price_for_fuel( self.data[self.demandkey], fuel, new_price )

            
            self.demand = Demand(self.data)
            quant_hat = self.demand.fuel_demand
            new_price = {ft[self.keyfuel]: find_price_for_fuel( fuel_type = ft[self.keyfuel], data = self.techs ) for ft in self.fuels}
            self.supply = Supply(self.data)
            
            price_hat = self.supply.tot_price
            
            for ft, val in new_price.items():
                self.results[self.reskeys[0]][ft].append(val)
            for ft, val in quant_hat.items():    
                self.results[self.reskeys[1]][ft].append(val)
            for ft, val in price_hat.items():
                self.results[self.reskeys[2]][ft].append(val)          
                
            #TODO (done here oct 13 pm, check price diff vals, check label fuck up)
            for f in self.info["fuels"]:
                last_price = self.results["dem_price_by_fuel"][f][-1]    
                cur_price = self.results["sup_price_by_fuel"][f][-1]
                price_diff[f] = np.abs(last_price - cur_price)
                price_thres[f] = cur_price * relative_tol
                ge_thres[f] = price_diff[f] > price_thres[f]
                
            self.info["price_diff"].append( price_diff )
            n_iter += 1

                


In [ ]:
sd = SupplyDemand(data)
sd.iterate()

In [ ]:
sd.results

In [ ]:
demPrice = sd.results["dem_price_by_fuel"]["electricity"]
supPrice = sd.results['sup_price_by_fuel']["electricity"]
quants = sd.results["quant_by_fuel"]["electricity"]

In [ ]:
sd.info